In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_curve, auc, roc_auc_score,
                             confusion_matrix)
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import pickle
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from imblearn.over_sampling import SMOTE, RandomOverSampler

In [2]:
df = pd.read_csv('given_dataset.csv')
df.drop('id', axis = 1, inplace = True)
df.dropna()
df.drop_duplicates()
df.head()

,A,B,C,D,E,F,G,T
0,8.341749e+07,2,124,4,0.144868,3.091977e-01,22,1
1,1.379494e+08,2,5,34,0.117274,9.492332e-03,0,1
2,1.293132e+08,2,324,18,0.006898,6.352157e-07,2,1
3,1.028544e+08,2,202,4,0.000000,8.980000e-02,0,1
4,2.784674e+09,2,281,56,0.000000,3.288574e-04,0,1


In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['T'].value_counts()
# Cases of 0 is just a measly 1.44% 

In [ ]:
# df['B'].value_counts()

In [ ]:
# main = df.index
# ones_B_2 = df[(df['B'] == 2) & (df['T'] == 1)].sample(n = 30000).index
# df_1 = df.loc[ones_B_2]
# ones_B_1 = df[(df['B'] == 1) & (df['T'] == 1)].sample(n = 10000).index
# df_2 = df.loc[ones_B_1]
# ones = df.loc[ones_B_2.union(ones_B_1)]
# general = df[df['T'] == 1].index
# first = general[~general.index.isin(df_1.index+df_2.index)]
# zeros = df[df['T'] == 0].index
# second = df.loc[zeros]

# df_ = first.join(second)
# temp = ones_B_2.union(ones_B_1)
# temp_2 = zeros.union(ones)
# first = df.loc[temp]
# second = df.loc[temp_2]

# df_.head()
# inter = pd.merge(union, second, how = 'inner')
# ans = pd.merge(union, second, how = 'outer')
# df_final = ans[ans.isnull().any(axis = 1)]
# df_final.head()
ones = df[df['T'] == 1].sample(n = 750000).index
zeros = df[df['T'] == 0].index

df = df.loc[ones.union(zeros)]

In [5]:
# X = df[['A', 'B', 'C', 'D', 'E', 'F', 'G']]
# y = df['T']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

# RUS
X_maj = df[df['T'] == 1]
X_min = df[df['T'] == 0]
X_maj_rus = resample(X_maj, replace = False, n_samples = 2*len(X_min), random_state = 42)
X_rus = pd.concat([X_maj_rus, X_min])
X_train = X_rus.drop(['T'], axis = 1)
y_train = X_rus['T']
# sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
# ROS
# ros = RandomOverSampler(sampling_strategy=1.0)
# X_ros, y_ros = ros.fit_resample(X_train, y_train)

# SMOTE
# smote = SMOTE(sampling_strategy=1.0, k_neighbors=2)
# X_train, y_train = smote.fit_resample(X_train, y_train)

# Wilson's ENN
# enn = EditedNearestNeighbours()
# X_enn, y_enn = enn.fit_resample(X_train, y_train)

# Tomek Links
# tomek = TomekLinks()
# X_tomek, y_tomek = tomek.fit_resample(X_train, y_train)

In [6]:
cv_method = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=42)
model_KNN = KNeighborsClassifier()
params_KNN = {'n_neighbors': [5, 7, 9], 
              'p': [1, 2, 3]}
gs_KNN = GridSearchCV(estimator=model_KNN, 
                      param_grid=params_KNN, 
                      cv=cv_method,
                      verbose=1, 
                      scoring='roc_auc')

In [7]:
gs_KNN.fit(X_train, y_train)

Fitting 30 folds for each of 9 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:  1.7min finished


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=42),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 7, 9], 'p': [1, 2, 3]},
             scoring='roc_auc', verbose=1)

In [8]:
gs_KNN.best_params_

{'n_neighbors': 9, 'p': 1}

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 9, p = 1)

In [ ]:
knn.fit(X_tomek, y_tomek)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1111)

In [ ]:
def show_cm(y_true, y_pred, class_names=None, model_name=None):
#     Show confusion matrix
 
 cf = confusion_matrix(y_true, y_pred)
 plt.imshow(cf, cmap=plt.cm.Blues)
 if model_name:
   plt.title('Confusion Matrix: {}'.format(model_name))
 else:
   plt.title('Confusion Matrix')
   plt.ylabel('True Label')
   plt.xlabel('Predicted Label')
 if class_names:
   tick_marks = np.arange(len(class_names))
   plt.xticks(tick_marks, class_names)
   plt.yticks(tick_marks, class_names)
 else:
   class_names = set(y_true)
   tick_marks = np.arange(len(class_names))
   plt.xticks(tick_marks, class_names)
   plt.yticks(tick_marks, class_names)
 thresh = cf.max() / 2.0
 for i, j in itertools.product(range(cf.shape[0]),
                               range(cf.shape[1])):
   plt.text(j, i, cf[i, j],
            horizontalalignment='center',
            color='white' if cf[i, j] > thresh else 'black',
            )
 plt.colorbar()

In [9]:
df_test = pd.read_csv('to_predict.csv')
df_test.drop_duplicates()
df_test.dropna()
op = df_test[['id']]
df_test.drop('id', axis = 1, inplace = True)
df_test.head()

,A,B,C,D,E,F,G
0,3.048551e+08,2,126,40,0.000000,0.673500,0
1,8.306319e+07,2,272,18,0.006898,0.342363,0
2,1.773707e+09,2,121,2,0.000000,0.673500,0
3,1.000000e+08,1,121,0,0.000000,0.673500,0
4,3.300000e+07,1,71,4,0.000000,0.168375,0


In [10]:
X = df_test[['A', 'B', 'C', 'D', 'E', 'F', 'G']]
ans = gs_KNN.predict(X)

In [11]:
op.insert(1, 'T', ans, True)
op.head()

,id,T
0,2041687,1
1,2041688,1
2,2041689,1
3,2041690,1
4,2041691,1


In [12]:
op.to_csv('submission_kNN_RUS.csv', index = False)